# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Geting CSV data to DataFrame
CitiesDF = pd.read_csv("Resources/cities.csv")
CitiesDF.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,iqaluit,75,CA,1635630291,69,63.7506,-68.5145,37.13,11.01
1,1,hermanus,58,ZA,1635630291,87,-34.4187,19.2345,61.90,8.39
2,2,mataura,100,NZ,1635630292,96,-46.1927,168.8643,49.66,12.59
3,3,albany,93,US,1635630292,90,42.6001,-73.9662,57.78,3.33
4,4,butaritari,24,KI,1635630293,73,3.0707,172.7902,82.38,6.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
#Configure gmaps with API key.
gmaps.configure(api_key = g_key)

#Droping the NaN values
CitiesDF = CitiesDF.dropna()

#Convert Humidity to Float from String Data Type. 
Humidity = CitiesDF["Humidity"].astype(float)

#Geting Lats and Longs.
Location = CitiesDF[["Lat","Lng"]].astype(float)

#Create Humidity Heatmap Layer.
MapFig = gmaps.figure()

HeatLayer = gmaps.heatmap_layer(Location, weights=Humidity, 
                                 dissipating=False, max_intensity=max(CitiesDF["Humidity"]),
                                 point_radius = 2)
MapFig.add_layer(HeatLayer)

MapFig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
# A max temperature lower than 80 degrees but higher than 70.
FilteredWeatherDF= CitiesDF.loc[(CitiesDF["Max Temp"]<80)&(CitiesDF["Max Temp"]>70)]
# Wind speed less than 10 mph.
FilteredWeatherDF = FilteredWeatherDF.loc[FilteredWeatherDF["Wind Speed"]<10]
# Zero cloudiness
FilteredWeatherDF = FilteredWeatherDF.loc[FilteredWeatherDF["Cloudiness"]==0]
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
FilteredWeatherDF=FilteredWeatherDF.dropna()
FilteredWeatherDF

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
25,25,norman wells,0,US,1635630307,30,35.2226,-97.4395,74.86,1.99
41,41,quelimane,0,MZ,1635630314,77,-17.8786,36.8883,71.15,5.93
141,141,fayaoue,0,NC,1635630366,71,-20.6500,166.5333,74.88,8.05
268,268,salalah,0,OM,1635630427,65,17.0151,54.0924,78.89,4.61
301,301,isangel,0,VU,1635630444,73,-19.5500,169.2667,77.11,5.75
325,325,hervey bay,0,US,1635630456,33,33.4982,-93.7771,72.05,7.45
374,374,koungou,0,YT,1635630480,83,-12.7336,45.2042,74.91,3.44
398,398,shahdadkot,0,PK,1635630494,18,27.8477,67.9055,73.26,1.92
419,419,karratha,0,AU,1635630506,68,-20.7377,116.8463,72.84,4.41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
# Store into variable named hotel_df.
hotel_df = FilteredWeatherDF
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    BaseUrl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(BaseUrl, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 25: norman wells.
Closest hotel is Montford Inn.
------------
Retrieving Results for Index 41: quelimane.
Closest hotel is Hotel Flamingo.
------------
Retrieving Results for Index 141: fayaoue.
Closest hotel is Chez Dydyce Guesthouse.
------------
Retrieving Results for Index 268: salalah.
Closest hotel is HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel.
------------
Retrieving Results for Index 301: isangel.
Closest hotel is Tanna Lodge.
------------
Retrieving Results for Index 325: hervey bay.
Missing field/result... skipping.
------------
Retrieving Results for Index 374: koungou.
Closest hotel is Villa Netibor.
------------
Retrieving Results for Index 398: shahdadkot.
Closest hotel is Malak Hotel shahdadkot.
------------
Retrieving Results for Index 419: karratha.
Closest hotel is ibis Styles Karratha.
------------


In [34]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
25,25,norman wells,0,US,1635630307,30,35.2226,-97.4395,74.86,1.99,Montford Inn
41,41,quelimane,0,MZ,1635630314,77,-17.8786,36.8883,71.15,5.93,Hotel Flamingo
141,141,fayaoue,0,NC,1635630366,71,-20.6500,166.5333,74.88,8.05,Chez Dydyce Guesthouse
268,268,salalah,0,OM,1635630427,65,17.0151,54.0924,78.89,4.61,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
301,301,isangel,0,VU,1635630444,73,-19.5500,169.2667,77.11,5.75,Tanna Lodge
325,325,hervey bay,0,US,1635630456,33,33.4982,-93.7771,72.05,7.45,NaN
374,374,koungou,0,YT,1635630480,83,-12.7336,45.2042,74.91,3.44,Villa Netibor
398,398,shahdadkot,0,PK,1635630494,18,27.8477,67.9055,73.26,1.92,Malak Hotel shahdadkot
419,419,karratha,0,AU,1635630506,68,-20.7377,116.8463,72.84,4.41,ibis Styles Karratha


In [39]:
# NOTE: Do not change any of the code in this cell
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
MapFig.add_layer(markers)
MapFig.add_layer(hotel_layer)
MapFig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Add marker layer ontop of heat map


# Display figure
